# The Structure & Interpretation of Computer Programs

## Chapter 1

> We are about to study the idea of a computational process. Computational processes are abstract beings
that inhabit computers. As they evolve, processes manipulate other abstract things called data. The
evolution of a process is directed by a pattern of rules called a program

> If Lisp is not a mainstream language, why are we using it as the framework for our discussion of
programming? Because the language possesses unique features that make it an excellent medium for
studying important programming constructs and data structures and for relating them to the linguistic
features that support them. The most significant of these features is the fact that Lisp descriptions of
processes, called procedures, can themselves be represented and manipulated as Lisp data.

> A critical aspect of a programming language is the means it provides for using names to refer to
computational objects. We say that the name identifies a variable whose value is the object [...] It should be clear that the possibility of associating values with symbols and later retrieving them means
that the interpreter must maintain some sort of memory that keeps track of the name-object pairs. This
memory is called the environment (more precisely the global environment, since we will see later that a
computation may involve a number of different environments)


In [4]:
(define π 3.14159)  ; write \pi+TAB to make π
(define size 2)

(* π size)

6.28318

### Evaluating Combinations

**Combinations** are lists of expressions within parantheses in order to denote procedure application. The leftmost element is the **operator** (prefix notation is used) and the remaining elements are the **operands**.

To evaluate a combination we need to (a) evaluate the operands (which can be themselves combinations) and apply the operator to the values from the previous computed operands. This is a strict evaluation, SICP calls it *applicative-order evaluation* (opposed to lazy evaluation, or *normal-order evaluation* to be discussed on chapters 3 and 4).

This rule does not apply to `define`, so `(define size 2)` is not a combination. These exceptions are called **special forms**.

`define` can be used to define procedures

In [12]:
(define (square x) (* x x)) ; (define (<name> <formal parameters>) <body>)

(display (square 5))
(newline)
(display (square (square 5)))

25
625

### Conditional Expressions and Predicates

Lisp allows for conditional expressions:

In [16]:
(define (abs x)
  (cond ((> x 0) x)    ; The first expression in each pair is a predicate
        ((= x 0) 0)    
        ((< x 0) (- x))))

; or

(define (abs x)
  (if (< x 0) (- x) x))

(abs -5)

5

The word predicate is used for procedures that return true or false, as well as for expressions that evaluate
to true or false.

We can combine predicates with `and`, `or` , `not`. The `and` and `or` are special forms since it is not guaranteed that all operands will be evaluated.

In [18]:
(define (>= x y)
  (or (> x y) (= x y)))

(>= 4 2)   ; #t is True, #f is False

#t

### Example: Square Roots by Newton's Method

> Procedures, as introduced above, are much like ordinary mathematical functions. They specify a value that
is determined by one or more parameters. But there is an important difference between mathematical
functions and computer procedures. Procedures must be effective.

> The contrast between function and procedure is a reflection of the general distinction between describing
properties of things and describing how to do things, or, as it is sometimes referred to, the distinction
between declarative knowledge and imperative knowledge. In mathematics we are usually concerned with
declarative (what is) descriptions, whereas in computer science we are usually concerned with imperative
(how to) descriptions.

Newton's method can be described as:

In [23]:
(define (average x y)
  (/ (+ x y) 2))

(define (good-enough? guess x)
  (< (abs (- (square guess) x)) 0.001))

(define (improve guess x)
  (average guess (/ x guess)))

(define (sqrt-iter guess x)
  (if (good-enough? guess x)
     guess
     (sqrt-iter (improve guess x) x)))

(define (sqrt x)
  (sqrt-iter 1.0 x))

(sqrt 2)

1.4142156862745097

In this example we could internalize all auxiliary procedures into the main `sqrt`:

In [ ]:
(define (sqrt x)
  ; x does not need to be a parameter below given its lexical scoping is bound to sqrt
  (define (good-enough? guess)
    (< (abs (- (square guess) x)) 0.001))
  (define (improve guess)
    (average guess (/ x guess)))
  (define (sqrt-iter guess)
    (if (good-enough? guess)
        guess
        (sqrt-iter (improve guess))))
  (sqrt-iter 1.0))

### Linear Recursion and Iteration

In [7]:
; Recursive solution
(define (factorial n)
  (if (= n 0)
      1
      (* n (factorial (- n 1)))))

(factorial 20)

2432902008176640000

The iterative solution uses extra parameters to represent state of needed variables:

In [8]:
; Iterative solution
(define (factorial n)
  (define (iter product counter)
    (if (> counter n)
        product
        (iter (* counter product)
              (+ counter 1))))
  (iter 1 1))    

(factorial 20)

2432902008176640000

> The substitution model reveals a shape of expansion followed by contraction, [...] The expansion occurs as the process builds up a chain of deferred operations (in this case, a chain of multiplications). The contraction occurs as the operations are actually performed. This type of process, characterized by a chain of *deferred operations*, is called a *recursive process*.

> By contrast, the second process does not grow and shrink. At each step, all we need to keep track of, for
any n, are the current values of the variables product, counter, and max-count. We call this an
*iterative process*. In general, an iterative process is one whose state can be summarized by a fixed number
of *state variables*, together with a fixed rule that describes how the state variables should be updated as the
process moves from state to state and an (optional) end test that specifies conditions under which the
process should terminate. In computing n!, the number of steps required grows linearly with n. Such a
process is called a *linear iterative process*.

> The contrast between the two processes can be seen in another way. In the iterative case, the program
variables provide a complete description of the state of the process at any point. If we stopped the
computation between steps, all we would need to do to resume the computation is to supply the interpreter
with the values of the three program variables. Not so with the recursive process. In this case there is some
additional ``hidden'' information, maintained by the interpreter and not contained in the program variables,
which indicates ``where the process is'' in negotiating the chain of deferred operations. The longer the
chain, the more information must be maintained.

> In contrasting iteration and recursion, we must be careful not to confuse the notion of a recursive process
with the notion of a recursive *procedure*. When we describe a procedure as recursive, we are referring to
the syntactic fact that the procedure definition refers (either directly or indirectly) to the procedure itself.
But when we describe a process as following a pattern that is, say, linearly recursive, we are speaking
about how the process evolves, not about the syntax of how a procedure is written. It may seem disturbing
that we refer to a recursive procedure such as `fact-iter` as generating an iterative process. However,
the process really is iterative: Its state is captured completely by its three state variables, and an interpreter
need keep track of only three variables in order to execute the process.

> One reason that the distinction between process and procedure may be confusing is that most
implementations of common languages (including Ada, Pascal, and C) are designed in such a way that the
interpretation of any recursive procedure consumes an amount of memory that grows with the number of
procedure calls, even when the process described is, in principle, iterative. As a consequence, these
languages can describe iterative processes only by resorting to special-purpose ``looping constructs'' such
as do, repeat, until, for, and while. The implementation of Scheme [...] does not share this defect. It will execute an iterative process in constant space, even if the iterative process is described by a recursive procedure. An implementation with this property is called *tail-recursive*. With a tail-recursive implementation, iteration can be expressed using the ordinary procedure call mechanism, so that special iteration constructs are useful only as syntactic sugar

### Tree Recursion

In [15]:
(define (fib n)
  (cond ((= n 0) 0)
        ((= n 1) 1)
        (else (+ (fib (- n 1))
                 (fib (- n 2))))))

> This procedure is instructive as a prototypical tree recursion, but it is a terrible way to compute Fibonacci
numbers because it does so much redundant computation

In [60]:
; Iterative solution
(define (fib n)
  (define (fib-iter a b count)
    (if (= count 0)
        b
        (fib-iter (+ a b) a (- count 1))))
  (fib-iter 1 0 n))

(fib 100)

354224848179261915075

> One should not conclude from this that tree-recursive processes are useless. When we consider processes
that operate on hierarchically structured data rather than numbers, we will find that tree recursion is a
natural and powerful tool. But even in numerical operations, tree-recursive processes can be useful in
helping us to understand and design programs. For instance, although the first fib procedure is much less
efficient than the second one, it is more straightforward, being little more than a translation into Lisp of the
definition of the Fibonacci sequence. To formulate the iterative algorithm required noticing that the
computation could be recast as an iteration with three state variables.

In [23]:
; This can be made with lists, but we're still in chapter 1
(define (count-change amount)
  (cc amount 5))

(define (cc amount kinds-of-coins)
  (cond ((= amount 0) 1)
        ((or (< amount 0) (= kinds-of-coins 0)) 0)
        (else (+ (cc amount
                     (- kinds-of-coins 1))
                 (cc (- amount
                        (first-denomination kinds-of-coins))
                     kinds-of-coins)))))

(define (first-denomination kinds-of-coins)
  (cond ((= kinds-of-coins 1) 1)
        ((= kinds-of-coins 2) 5)
        ((= kinds-of-coins 3) 10)
        ((= kinds-of-coins 4) 25)
        ((= kinds-of-coins 5) 50)))

(count-change 6) ; 6*1 or 5+1

2

Computing Pascal triangle values:

In [24]:
(define (pascal r c) 
  (if (or (= c 1) (= c r)) 
     1 
    (+ (pascal (- r 1) (- c 1)) (pascal (- r 1) c)))) 

(pascal 5 3) ; row 5 is 1 4 6 4 1

6

### Example: Testing for Primality

Deterministic $\mathcal{O}(\sqrt{n})$ algorithm

In [27]:
(define (square x) 
  (* x x)) 

(define (divides? a b)
  (= (remainder b a) 0))

(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n) n)
        ((divides? test-divisor n) test-divisor)
        (else (find-divisor n (+ test-divisor 1)))))

(define (smallest-divisor n)
  (find-divisor n 2))

(define (prime? n)
  (= n (smallest-divisor n)))

(prime? 71)

#t

Probabilistic $\mathcal{O}(\log n)$ algorithm using Fermat's little theorem.

In [30]:
; computes the exponential of a number modulo another number
(define (expmod base exponent m)
  (cond ((= exponent 0) 1)
        ((even? exponent)
         (remainder (square (expmod base (/ exponent 2) m))
                    m))
        (else
         (remainder (* base (expmod base (- exponent 1) m))
                    m))))    

(expmod 2 10 5) ; 2^10 mod 5

4

The Fermat test is performed by choosing at random a number a between 1 and n - 1 inclusive and checking whether the remainder modulo n of the nth power of a is equal to a.

In [35]:
(define (fermat-test n)
  (define (try-it a)
    (= (expmod a n n) a))
  (try-it (+ 1 (random (- n 1)))))

(define (fast-prime? n times)
  (cond ((= times 0) #t)
        ((fermat-test n) (fast-prime? n (- times 1)))
        (else #f)))

(fast-prime? 71 10)

#t

In [50]:
; checking how long the computation takes

(define (timed-prime-test n)
  (newline)
  (display n)
  (start-prime-test n (current-time)))

(define (start-prime-test n start-time)
  (if (fast-prime? n 10) ; using 10 checks
      (report-prime (- (current-time) start-time))))

(define (report-prime elapsed-time)
  (display " *** is prime *** ")
  (display (* elapsed-time 1000))
  (display " ms"))

(timed-prime-test 723113)


723113 *** is prime *** 132.63988494873047 ms

## Formulating Abstractions with Higher-Order Procedures

Procedures that manipulate procedures are called *higher-order procedures*.

Consider a procedure that sums numbers from a to b, but before we must apply some transformation to those numbers. If the language has the ability to pass that transformation to the procedure, we can create just one procedure that captures this task.

In [63]:
(define (sum-map f a b)
  (define (sum-map-iter f a b sum)
    (cond ((> a b) sum)
          (else (sum-map-iter f (+ a 1) b (+ sum (f a))))))
  (sum-map-iter f a b 0))

(display (sum-map square 1 3))
(newline)
(display (sum-map fib 1 10))

14
143

We can still generalize more by defining the next step (instead of being just i+1)

In [66]:
(define (sum-map-next f a next b)
  (define (sum-map-iter f a b sum)
    (cond ((> a b) sum)
          (else (sum-map-iter f (next a) b (+ sum (f a))))))
  (sum-map-iter f a b 0))

(define (skip-one n) (+ n 2))
(sum-map-next square 1 skip-one 10) ; 1^2 + 3^2 + 5^2 + 7^2 + 9^2

165

### Constructing Procedures Using Lambda

Scheme has anonymous functions (of course!)

In [67]:
(sum-map-next (lambda (x) (* x x)) 1 skip-one 10)

165

Lambdas can be used to define local variables to work inside a procedure:

In [71]:
; f(x,y) = x(1+xy)^2 + y(1-y) + (1+xy)(1-y)  so lots of repetitions, let's make a=1+xy and b=1-y
(define (f x y)
  ((lambda (a b)
   (+ (* x (square a)) (* y b) (* a b)))
    (+ 1 (* x y))
    (- 1 y))
  )

(f 3 5)

684

Since this need is so common, there's a special form for it, `let`:

In [74]:
(define (f x y)
  (let ((a (+ 1 (* x y)))
        (b (- 1 y)))
  (+ (* x (square a))
     (* y b)
     (* a b))))

(f 3 5)

684

### Procedures as Returned Values

In [80]:
(define (inc-factory n)
  (lambda (x) (+ x n)))

(define inc  (inc-factory 1))
(define inc2 (inc-factory 2))

(display (inc  1))
(newline)
(display (inc2 1))

2
3

> As programmers, we should be alert to opportunities to identify the underlying abstractions in our
programs and to build upon them and generalize them to create more powerful abstractions. This is not to
say that one should always write programs in the most abstract way possible; expert programmers know
how to choose the level of abstraction appropriate to their task. But it is important to be able to think in
terms of these abstractions, so that we can be ready to apply them in new contexts. The significance of
higher-order procedures is that they enable us to represent these abstractions explicitly as elements in our
programming language, so that they can be handled just like other computational elements.

---

SCIP solutions at http://community.schemewiki.org/?SICP-Solutions